Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

In [184]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import numpy as np
import pandas as pd

### Load Competition Data

In [132]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [133]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [134]:
Y_train = train['ratingCategory']
X_train = train['description']

In [135]:
X_train

0       \nSometimes, when whisky is batched, a few lef...
1       \nAn uncommon exclusive bottling of a 6 year o...
2       \nThis release is a port version of Amrut’s In...
3       \nThis 41 year old single cask was aged in a s...
4       \nQuite herbal on the nose, with aromas of dri...
                              ...                        
4082    \nWhat lies beneath the surface of Dewar’s? He...
4083    \nAfter 6 to 7 years of maturation in bourbon ...
4084    \nBright, delicate, and approachable. While no...
4085    \nI’m calling this the pitmaster’s dram: the n...
4086    \nSpicy sultanas, greengage plums, toffee, and...
Name: description, Length: 4087, dtype: object

In [165]:
# Cleaning Data
text_ex = X_train[50]
text_ex

'\nOne of a series of single cask, wine-finished Caol Ilas. This one is finished in a claret (Bordeaux) wine cask. The claret adds some bright red fruit notes (raspberry, red currant) and red licorice to accompany the smoke, brine, olive, and vanilla-laced malt. I have tasted some claret/Bordeaux wood-finished whiskies where the wine was too dominant, but this one achieves a nice balance. \r\n'

In [162]:
import re

In [192]:
def clean_text(text):
    lower = re.sub('[^a-zA-Z 0-9]', '', text)
    lower = lower.lower()
#     nlp = spacy.load("en_core_web_lg")
#     doc = nlp(lower)
#     tokens = []
#     for token in doc:
#         if (token.is_stop != True) and (token.is_punct != True):
#             tokens.append(token.lemma_.lower())
    return lower

In [193]:
X_train = X_train.apply(clean_text)
X_train

0       sometimes when whisky is batched a few leftove...
1       an uncommon exclusive bottling of a 6 year old...
2       this release is a port version of amruts inter...
3       this 41 year old single cask was aged in a she...
4       quite herbal on the nose with aromas of dried ...
                              ...                        
4082    what lies beneath the surface of dewars here t...
4083    after 6 to 7 years of maturation in bourbon ca...
4084    bright delicate and approachable while not a s...
4085    im calling this the pitmasters dram the nose h...
4086    spicy sultanas greengage plums toffee and new ...
Name: description, Length: 4087, dtype: object

### Define Pipeline Components

In [136]:
vect = TfidfVectorizer(stop_words="english")
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [137]:
parameters = {
    # exclude tokens that appear in range
    "vect__max_df": (0.9, 1.0),
    "vect__min_df": (2, 10),
    "vect__max_features": [500, 1000],
    # included parameters for our ML model
    # it would be wise to include 1000
    "clf__n_estimators": [10, 100],
    "clf__max_depth": [None, 10]
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=4, verbose=1)

In [138]:
grid_search.fit(X_train, Y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(stop_words='english')),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=4,
             param_grid={'clf__max_depth': [None, 10],
                         'clf__n_estimators': [10, 100],
                         'vect__max_df': (0.9, 1.0),
                         'vect__max_features': [500, 1000],
                         'vect__min_df': (2, 10)},
             verbose=1)

In [139]:
grid_search.best_score_

0.7401513677866127

In [140]:
grid_search.best_params_

{'clf__max_depth': None,
 'clf__n_estimators': 100,
 'vect__max_df': 1.0,
 'vect__max_features': 1000,
 'vect__min_df': 2}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [126]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [127]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [128]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [129]:
subNumber = 0

In [131]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [194]:
from sklearn.decomposition import TruncatedSVD

In [195]:
vect = TfidfVectorizer()
svd = TruncatedSVD()
clf = RandomForestClassifier()
lsi = Pipeline([('vect', vect),
               ('svd', svd)])

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [196]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20),
    'clf__n_estimators': (100, 1000)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(X_train, Y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('lsi',
                                        Pipeline(steps=[('vect',
                                                         TfidfVectorizer()),
                                                        ('svd',
                                                         TruncatedSVD())])),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=4,
             param_grid={'clf__max_depth': (5, 10, 15, 20),
                         'clf__n_estimators': (100, 1000),
                         'lsi__svd__n_components': [10, 100, 250],
                         'lsi__vect__max_df': (0.75, 1.0)},
             verbose=1)

In [197]:
grid_search.best_score_

0.7296322343357684

In [198]:
grid_search.best_params_

{'clf__max_depth': 20,
 'clf__n_estimators': 1000,
 'lsi__svd__n_components': 10,
 'lsi__vect__max_df': 0.75}

### Make a Submission File

In [146]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [147]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [148]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [150]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [199]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [200]:
# Continue Word Embedding Work Here
nlp = spacy.load("en_core_web_lg")

def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [201]:
X_train_2 = get_word_vectors(X_train)
X_test_2 = get_word_vectors(X_test)

NameError: name 'X_test' is not defined

In [ ]:
rfc = RandomForestClassifier(oob_score=True)

In [ ]:
rfc.fit(X_train, Y_train);

In [ ]:
rfc.score(X_train, Y_train)

In [ ]:
rfc.oob_score_

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?